In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import time

### 1. Model Architecture

In [27]:
def createModel():
  model = Sequential()
  model.add(Dense(64, activation='relu', input_shape=(7*7*1280,)))
  model.add(Dense(8, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.001), metrics=["accuracy"])

  return model



### 2. Training 

In [29]:
# Euclidean distance
def linear_diff(np1, np2):
  np1 = np1.flatten()
  np2 = np2.flatten()

  return np1 - np2

In [32]:
features = np.load("./train_X.npy")
trainY = np.load("./train_y.npy")

print(features.shape)
trainX = []
for i in range(0, len(features)):
  trainX.append(linear_diff(features[i][0], features[i][1]))

trainX = np.array(trainX)
print(trainX.shape)
print(trainY.shape)


# Shuffle
indices = np.arange(trainX.shape[0])
np.random.shuffle(indices)
trainX = trainX[indices]  
trainY = trainY[indices]




(156, 2, 7, 7, 1280)
(156, 62720)
(156,)


In [41]:
model = createModel()

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 64)                4014144   
                                                                 
 dense_11 (Dense)            (None, 8)                 520       
                                                                 
 dense_12 (Dense)            (None, 1)                 9         
                                                                 
Total params: 4,014,673
Trainable params: 4,014,673
Non-trainable params: 0
_________________________________________________________________


In [34]:
modelID = int(time.time())
method = "flatten&eudlidian"

print("modelID: ", modelID)

tensorboard_callback = keras.callbacks.TensorBoard(log_dir=f"./logs/{method}/" + str(modelID), histogram_freq=1)

checkpoint = ModelCheckpoint("./models/" + str(modelID) + ".h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

early = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1, mode='auto')

callbacks_list = [checkpoint, early, tensorboard_callback]

history = model.fit(trainX, trainY, batch_size=32, epochs=100, validation_split=0.2, callbacks=callbacks_list, shuffle=True)

modelID:  1703800271
Epoch 1/100
4/4 [==============================] - ETA: 0s - loss: 7.2461 - accuracy: 0.5565
Epoch 1: val_loss improved from inf to 21.45121, saving model to ./models/1703800271.h5
4/4 [==============================] - 3s 610ms/step - loss: 7.2461 - accuracy: 0.5565 - val_loss: 21.4512 - val_accuracy: 0.5938
Epoch 2/100
4/4 [==============================] - ETA: 0s - loss: 4.0933 - accuracy: 0.8226
Epoch 2: val_loss improved from 21.45121 to 14.45243, saving model to ./models/1703800271.h5
4/4 [==============================] - 2s 489ms/step - loss: 4.0933 - accuracy: 0.8226 - val_loss: 14.4524 - val_accuracy: 0.6562
Epoch 3/100
4/4 [==============================] - ETA: 0s - loss: 0.8945 - accuracy: 0.9355
Epoch 3: val_loss improved from 14.45243 to 12.60285, saving model to ./models/1703800271.h5
4/4 [==============================] - 2s 525ms/step - loss: 0.8945 - accuracy: 0.9355 - val_loss: 12.6029 - val_accuracy: 0.6875
Epoch 4/100
4/4 [===================

### Testing

In [43]:
# Load best model
model = keras.models.load_model("./models/" + str(modelID) + ".h5")

In [44]:
textY = np.load("./test_y.npy") 
textX = []
features = np.load("./test_X.npy")
for i in range(0, len(features)):
  textX.append(linear_diff(features[i][0], features[i][1]))

textX = np.array(textX)

print(textX.shape)
print(textY.shape)

score = model.evaluate(textX, textY, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


(40, 62720)
(40,)
Test loss: 2.172772169113159
Test accuracy: 0.824999988079071
